In [ ]:
#!pip install bs4
import bs4
import requests
import sys
import pandas as pd
import re
from matplotlib import pyplot as plt
import seaborn as sns
import time
import random as ran
from scipy import stats
import numpy as np
from scipy.stats import t
from numpy.polynomial.polynomial import polyfit

### Scraping Movies Data from IMDb Website using Beautiful Soup

In [ ]:
def extractcolumns(col,b):   
    data_d = {}
    name = col.find('a')
    year = col.find('span',{'class': 'lister-item-year'})
    rating = col.find('div',{'class':'inline-block ratings-imdb-rating'})
    m_score = col.find('span',{'class':'metascore'})
    votes = col.find('span',{'name':'nv'})
    gross = cols[b].findAll('span',{'name':'nv'})[-1]
    genre = cols[b].find('span', class_='genre')
    director = col.find_all('p')[2]    
    data_d['Movie_Name'] = name.get_text() if name else None
    if year.contents:
        data_d['Relese_Year'] = str(year.contents[0][1:-1].replace('I) (','')).strip() if year else None
    data_d['Rating_Score'] = float(rating.get('data-value')) if rating else None
    data_d['Meta_Score'] = float(m_score.contents[0].strip()) if m_score else None
    data_d['Total_Votes'] = int(votes.get('data-value').replace(",","")) if votes else None
    data_d['Gross_USA'] = int(gross.get('data-value').replace(",","")) if gross else None
    data_d['Movie_Genre'] = str(genre.get_text().replace('\n','').replace('"','').strip()) if genre else None
    data_d['Director_Star'] = director.text.replace('\n','').replace('    ','') if director else None    
    return data_d


#Provide the Imdb Website from which data needs to be scraped in 'movie_url' and movies count required in 'target'. Years in the movie_url need to be changed according to the data

movie_dataset = [] 
movie_url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2016-01-01,2016-12-31&sort=boxoffice_gross_us,desc&start='
#Provide the 
target = 200
start = 0
end = 0 
count = 1 
tcount = target - end  
while tcount > 0:
    url = movie_url + str(count)
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')       
    cols = soup.findAll('div',{'class':'lister-item-content'}) 
    movie_dataset.extend([extractcolumns(cols[b],b) for b in range(len(cols))])          
    start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0].replace(",",""))
    end = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[1].split(" ")[0].replace(",",  ""))
    tcount = target - end       
    count = end + 1       
e_data = pd.DataFrame(movie_dataset)
e_data.to_csv('MovieTest.csv')
print("Extraction Completed")


In [ ]:
cd Movie_Dataset

In [ ]:
#Reading Datasets

df_inflation = pd.read_csv('Inflation_Data.csv')
df_movies2014 = pd.read_csv('Movies2014.csv')
df_movies2015 = pd.read_csv('Movies2015.csv')
df_movies2016 = pd.read_csv('Movies2016.csv')
df_movies2017 = pd.read_csv('Movies2017.csv')
df_movies2018 = pd.read_csv('Movies2018.csv')
df_movies2019 = pd.read_csv('Movies2019.csv')
df_movies2020 = pd.read_csv('Movies2020.csv')
df_movies2021 = pd.read_csv('Movies2021.csv')

In [ ]:
#Merging all the movie datasets of years 2014-2022 into one Merged_Movies Dataset 

files = ['Movies2014.csv','Movies2015.csv', 'Movies2016.csv',
         'Movies2017.csv', 'Movies2018.csv', 'Movies2019.csv', 'Movies2020.csv', 'Movies2021.csv']
df_list = []
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)
merged_df = pd.concat(df_list)
merged_df.to_csv('Merged_Movies.csv', index=False)
df_merged_c = pd.read_csv('Merged_Movies.csv')
df_merged_c.drop(df_merged_c.filter(regex="Unname"),axis=1, inplace=True)

In [ ]:
#Data Cleaning

#Function to clear all the strings present in year column (Cleaning Year Column)

def extract_year(x):
    match = re.search(r'\b\d{4}\b', str(x))
    if match:
        return int(match.group().replace('.0','').strip())
    return None

df_merged_c['Relese_Year'] = df_merged_c['Relese_Year'].astype(str).apply(extract_year)
df_merged_c['Relese_Year'] = df_merged_c['Relese_Year'].astype(str).str.replace('nan', '',regex=True).replace('','00')
df_merged_c['Relese_Year'] = df_merged_c['Relese_Year'].astype(str).str.replace('.', '',regex=True).str[:-1].astype('int64')
df_merged_c = df_merged_c[df_merged_c['Relese_Year'] != 0]
df_merged_c.dropna(inplace = True)
df_merged_c[['Director', 'Stars']] = df_merged_c['Director_Star'].str.split('|', expand = True)
df_merged_c['Director'] = df_merged_c['Director'].str.replace('Director:', '')
df_merged_c['Director'] = df_merged_c['Director'].str.replace('Directors:', '')
df_merged_c['Director'] = df_merged_c['Director'].str.replace('Stars:', '')
df_merged_c['Stars'] = df_merged_c['Stars'].str.replace('Stars:', '')
df_merged_c.drop(df_merged_c.filter(regex="Director_Star"),axis=1, inplace=True)
df_merged_c['Gross_USA'] = df_merged_c['Gross_USA'].astype('int64')
df_merged_c['Meta_Score'] = df_merged_c['Meta_Score'].astype('int64')
df_merged_c['Total_Votes'] = df_merged_c['Total_Votes'].astype('int64')

### Analysis-1 Trend Analysis on Gross Revenue of Movies Trended from 2014 to 2021

In [ ]:
df_analysis1 = df_merged_c.groupby('Relese_Year').agg({'Gross_USA':['sum']}).reset_index()
df_analysis1.columns = ['Relese_Year', 'Gross_Sum']
df_analysis1_1 = df_analysis1.sort_values(["Relese_Year"], ascending = False)

In [ ]:
plt.figure(figsize = (6,4))
p = sns.lineplot(data = df_analysis1_1, x = 'Relese_Year', y = 'Gross_Sum')
p.tick_params(labelsize = 8)
plt.title('Gross Revenue of Movies Trended from 2014 to 2021', fontsize = 15)
plt.xlabel('Year', fontsize = 9)
plt.ylabel('Total Gross Revenue', fontsize = 9)
plt.savefig("1Gross-Year.pdf", format="pdf", bbox_inches="tight")
plt.show()

### Analysis 2 Exploring the Relationship Between IMDb Votes and Box Office Revenue

In [ ]:
plt.figure(figsize=(5,5))
sns.regplot(data=df_merged_c.sample(frac=1), x='Total_Votes', y='Gross_USA', color = 'blue', line_kws={'color': 'red'}, scatter_kws={'color': 'blue'})

p.tick_params(labelsize = 8)
plt.title('Correlation Between IMDb Votes and Box Office Revenue', fontsize = 12)
plt.xlabel('Number of Votes on IMDb', fontsize = 10)
plt.ylabel('Total Gross Revenue of Movies ', fontsize = 10)
plt.savefig("2Votes-Gross.pdf", format="pdf", bbox_inches="tight")
plt.show()

#### Pearson Test 

In [ ]:
(r,p) = stats.pearsonr(df_merged_c.Total_Votes, df_merged_c.Gross_USA)
print('r =', r, 'p =', p)

### Analysis 3 Examining the Relationship Between Gross Revenue and Ratings

In [ ]:
expanded_list=[]
for row in df_merged_c.itertuples():
    genres=row.Movie_Genre.split(',')
    for genre in genres:
        expanded_list.append([row.Movie_Name, row.Relese_Year, row.Rating_Score, row.Meta_Score, row.Total_Votes, row.Gross_USA,genre.strip(),
       row.Director, row.Stars])  
df_inflation.columns = ['Relese_Year','amount', 'inflation_rate']
factor=df_inflation[df_inflation.Relese_Year==2023]['amount']
df_inflation['Relese_Year']=df_inflation['Relese_Year'].astype(float)
df_genres_expanded = pd.DataFrame(expanded_list, columns = ['Movie_Name', 'Relese_Year', 'Rating_Score', 'Meta_Score', 'Total_Votes', 'Gross_USA', 'Movie_Genre','Director', 'Stars'])
df_genres_inflated_expanded = pd.merge(df_genres_expanded, df_inflation, on = 'Relese_Year')
df_genres_expanded['Gross_USA']=df_genres_expanded['Gross_USA'].astype(float)*float(factor)/(df_genres_inflated_expanded.amount).astype(float)


In [ ]:
gross_ratings_data = df_genres_expanded.groupby('Rating_Score').agg({'Gross_USA': 'mean'}).reset_index()
sns.regplot(data=gross_ratings_data.sample(frac=1), x='Rating_Score', y='Gross_USA', scatter_kws={'s': 3}, line_kws={'color': 'red'})
plt.title('Correlation Between Ratings and Box Office Revenue', fontsize = 12)
plt.xlabel('Ratings', fontsize = 10)
plt.ylabel('Total Gross Revenue', fontsize = 10)
plt.savefig("4Gross-Rating.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
(r,p) = stats.pearsonr(gross_ratings_data.Rating_Score, gross_ratings_data.Gross_USA)
print('r =', r, 'p =', p)

### Analysis 4 Proportion of Movies by Genre: A Pie Chart

In [ ]:
expanded_list=[]
for row in df_merged_c.itertuples():
    genres=row.Movie_Genre.split(',')
    for genre in genres:
        expanded_list.append([row.Movie_Name, row.Relese_Year, row.Rating_Score, row.Meta_Score, row.Total_Votes, row.Gross_USA,genre.strip(),
       row.Director, row.Stars])    
df_genres_expanded = pd.DataFrame(expanded_list, columns = ['Movie_Name', 'Relese_Year', 'Rating_Score', 'Meta_Score', 'Total_Votes', 'Gross_USA', 'Movie_Genre','Director', 'Stars'])    
df_inflation.columns = ['Relese_Year','amount', 'inflation_rate']
factor=df_inflation[df_inflation.Relese_Year==2023]['amount']
df_inflation['Relese_Year']=df_inflation['Relese_Year'].astype(float)
df_genres_expanded = pd.DataFrame(expanded_list, columns = ['Movie_Name', 'Relese_Year', 'Rating_Score', 'Meta_Score', 'Total_Votes', 'Gross_USA', 'Movie_Genre','Director', 'Stars'])
df_genres_inflated_expanded = pd.merge(df_genres_expanded, df_inflation, on = 'Relese_Year')
df_genres_expanded['Gross_USA']=df_genres_expanded['Gross_USA'].astype(float)*float(factor)/(df_genres_inflated_expanded.amount).astype(float)
grp_mov_collection = df_genres_expanded.groupby(['Movie_Genre']).agg({'Gross_USA': 'mean','Total_Votes':'mean','Meta_Score':'mean','Rating_Score':'mean'}).reset_index()

In [ ]:
plt.figure(figsize=(7,7))
labels=grp_mov_collection['Movie_Genre'].unique()
plt.pie(data=grp_mov_collection, x='Gross_USA',autopct='%.0f%%' ,shadow=False)
plt.title('Proportion of Movies by Genre')
plt.legend(labels=labels)
plt.savefig("3genre-pie.pdf", format="pdf", bbox_inches="tight")
plt.show()

### Analysis 5 Comparing Director and Movie Ratings: A T-Test Analysis

In [ ]:
expanded_list1=[]
for row in df_merged_c.itertuples():
    directors=row.Director.split(',')
    for director in directors:
        expanded_list1.append([row.Movie_Name, row.Relese_Year, row.Rating_Score, row.Meta_Score, row.Movie_Genre, row.Total_Votes, row.Gross_USA, director.strip(), row.Stars])  
df_dir_expanded = pd.DataFrame(expanded_list1, columns = ['Movie_Name', 'Relese_Year', 'Rating_Score', 'Meta_Score', 'Total_Votes', 'Gross_USA', 'Movie_Genre','Director', 'Stars'])
df_5 = df_dir_expanded
df_5_g = df_5.groupby(['Director', 'Movie_Name']).agg({'Rating_Score':['mean']}).reset_index()
df_5_g.columns = ['Director','Movie_Name', 'Rating_Score' ]
df_5_g = df_5_g.dropna().sort_values('Rating_Score', ascending = False).replace('nan', '')

In [ ]:
sns.displot(data=df_5_g[df_5_g.Director.isin(['Tyler Perry', 'Christopher Nolan'])],
            x='Rating_Score', hue='Director', kind='kde',common_norm=False)
plt.savefig("5Director-Rating.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
display(f'Tyler Perry mean: {df_5_g[df_5_g.Director=="Tyler Perry"].Rating_Score.mean()}')
display(f'Christopher Nolan mean: {df_5_g[df_5_g.Director=="Christopher Nolan"].Rating_Score.mean()}')
display(f'Tyler Perry sd: {df_5_g[df_5_g.Director=="Tyler Perry"].Rating_Score.std()}')
display(f'Christopher Nolan sd: {df_5_g[df_5_g.Director=="Christopher Nolan"].Rating_Score.std()}')
display(stats.ttest_ind(df_5_g[df_5_g.Director == 'Tyler Perry'].Rating_Score, df_5_g[df_5_g.Director == 'Christopher Nolan'].Rating_Score))